In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, recall_score
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
import itertools
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
from dataloader_creator import CreatorDL
creator = CreatorDL(seed=42, bs=2048)

In [3]:
df_UNSW = creator.reader("NF-UNSW-NB15-v3")

df_train_UNSW, df_test_UNSW, df_val_UNSW = creator.splitter(df_UNSW)

train_loader_UNSW, test_loader_UNSW, val_loader_UNSW = creator.balancer(df_train_UNSW, df_test_UNSW, df_val_UNSW)

Processando a categoria: 'Benign'
  -> Treino: 1118865 | Teste: 559433 | Validação: 559433
Processando a categoria: 'Fuzzers'
  -> Treino: 16908 | Teste: 8454 | Validação: 8454
Processando a categoria: 'Exploits'
  -> Treino: 21374 | Teste: 10687 | Validação: 10687
Processando a categoria: 'Backdoor'
  -> Treino: 2329 | Teste: 1165 | Validação: 1165
Processando a categoria: 'Reconnaissance'
  -> Treino: 8537 | Teste: 4268 | Validação: 4269
Processando a categoria: 'Generic'
  -> Treino: 9825 | Teste: 4913 | Validação: 4913
Processando a categoria: 'DoS'
  -> Treino: 2990 | Teste: 1495 | Validação: 1495
Processando a categoria: 'Shellcode'
  -> Treino: 1190 | Teste: 595 | Validação: 596
Processando a categoria: 'Analysis'
  -> Treino: 613 | Teste: 306 | Validação: 307
Processando a categoria: 'Worms'
  -> Treino: 79 | Teste: 39 | Validação: 40

--- Base de Treino ---
Tamanho: 1182710 linhas
Categorias presentes: ['Benign' 'Exploits' 'Reconnaissance' 'Fuzzers' 'DoS' 'Generic' 'Backdoor'


In [4]:
df_BOT= creator.reader("NF-BoT-IoT-v3")

df_train_BOT, df_test_BOT, df_val_BOT = creator.splitter(df_BOT)

train_loader_BOT, test_loader_BOT, val_loader_BOT = creator.balancer(df_train_BOT, df_test_BOT, df_val_BOT)

Processando a categoria: 'Benign'
  -> Treino: 25994 | Teste: 12997 | Validação: 12998
Processando a categoria: 'DDoS'
  -> Treino: 3575441 | Teste: 1787720 | Validação: 1787721
Processando a categoria: 'DoS'
  -> Treino: 4017095 | Teste: 2008547 | Validação: 2008548
Processando a categoria: 'Reconnaissance'
  -> Treino: 847566 | Teste: 423783 | Validação: 423783
Processando a categoria: 'Theft'
  -> Treino: 807 | Teste: 404 | Validação: 404

--- Base de Treino ---
Tamanho: 8466903 linhas
Categorias presentes: ['DDoS' 'DoS' 'Reconnaissance' 'Benign' 'Theft']
Attack
DoS               4017095
DDoS              3575441
Reconnaissance     847566
Benign              25994
Theft                 807
Name: count, dtype: int64
-------------------------

--- Base de Teste ---
Tamanho: 4233451 linhas
Categorias presentes: ['DDoS' 'DoS' 'Reconnaissance' 'Benign' 'Theft']
Attack
DoS               2008547
DDoS              1787720
Reconnaissance     423783
Benign              12997
Theft            

In [5]:
df_CIC= creator.reader("NF-CICIDS2018-v3")

df_train_CIC, df_test_CIC, df_val_CIC = creator.splitter(df_CIC)

train_loader_CIC, test_loader_CIC, val_loader_CIC = creator.balancer(df_train_CIC, df_test_CIC, df_val_CIC)

Processando a categoria: 'Benign'
  -> Treino: 8757313 | Teste: 4378656 | Validação: 4378657
Processando a categoria: 'FTP-BruteForce'
  -> Treino: 193360 | Teste: 96680 | Validação: 96680
Processando a categoria: 'SSH-Bruteforce'
  -> Treino: 94237 | Teste: 47118 | Validação: 47119
Processando a categoria: 'DoS_attacks-GoldenEye'
  -> Treino: 30650 | Teste: 15325 | Validação: 15325
Processando a categoria: 'DoS_attacks-Slowloris'
  -> Treino: 18020 | Teste: 9010 | Validação: 9010
Processando a categoria: 'DoS_attacks-SlowHTTPTest'
  -> Treino: 52775 | Teste: 26387 | Validação: 26388
Processando a categoria: 'DoS_attacks-Hulk'
  -> Treino: 50038 | Teste: 25019 | Validação: 25019
Processando a categoria: 'DDoS_attacks-LOIC-HTTP'
  -> Treino: 144294 | Teste: 72147 | Validação: 72148
Processando a categoria: 'DDOS_attack-LOIC-UDP'
  -> Treino: 1725 | Teste: 862 | Validação: 863
Processando a categoria: 'DDOS_attack-HOIC'
  -> Treino: 516155 | Teste: 258078 | Validação: 258078
Processando 

In [6]:
train_loaders = [train_loader_UNSW, train_loader_BOT, train_loader_CIC]
test_loaders = [test_loader_UNSW, test_loader_BOT, test_loader_CIC]
val_loaders = [val_loader_UNSW, val_loader_BOT, val_loader_CIC]

In [37]:
INPUT_DIM = 32

class IDSBranchyNet(nn.Module):
    def __init__(self, input_dim=INPUT_DIM, num_classes=2):
        super(IDSBranchyNet, self).__init__()

        self.threshold_attack = nn.Parameter(torch.tensor([1.0986])) 
        self.threshold_normal = nn.Parameter(torch.tensor([1.0986])) 
        
        self.shared_layers = nn.Sequential(
            nn.Linear(input_dim, input_dim * 2),
            nn.ReLU(),
        )
        
        self.exit1_layers = nn.Sequential(
            nn.Linear(input_dim * 2, input_dim * 2),
            nn.LeakyReLU(),
            nn.Linear(input_dim * 2, num_classes)
        )
        
        self.exit2_layers = nn.Sequential(
            nn.Linear(input_dim * 2, 1024),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 2048),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, 2048),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, num_classes)
        )

    def get_threshold_attack(self):
        return torch.sigmoid(self.threshold_attack)

    def get_threshold_normal(self):
        return torch.sigmoid(self.threshold_normal)

    def forward_exit1(self, x):
        features = self.shared_layers(x)
        return self.exit1_layers(features)

    def forward_exit2(self, x):
        features = self.shared_layers(x)
        return self.exit2_layers(features)

model = IDSBranchyNet()

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [39]:
def train_model(model, train_loaders, val_loaders, epochs, lr, device, patience=15):

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.001, patience=7)
    
    model.to(device)
    
    criterion = nn.CrossEntropyLoss() 
    
    metrics = [
        'loss1_a', 'loss1_b', 'loss1_c', 'loss_ex1_avg',
        'loss2_a', 'loss2_b', 'loss2_c', 'loss_ex2_avg',
        'l_joint', 'total_loss'
    ]

    history = {
        'train': {k: [] for k in metrics},
        'val': {k: [] for k in metrics}
    }

    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    max_train_batches = max(len(l) for l in train_loaders) 
    train_iter_loaders = [itertools.cycle(l) if len(l) < max_train_batches else l for l in train_loaders]
    
    max_val_batches = max(len(l) for l in val_loaders)
    val_iter_loaders = [itertools.cycle(l) if len(l) < max_val_batches else l for l in val_loaders]

    def process_batch_logic(inputs, labels, model_ref, class_thresholds, crit):
        out1 = model_ref.forward_exit1(inputs)
        
        probs = F.softmax(out1, dim=1)
        conf, preds = torch.max(probs, dim=1)
        
        current_thresholds = class_thresholds[preds]
        
        mask_ex1 = conf > current_thresholds
        mask_ex2 = ~mask_ex1
    
        if mask_ex1.sum() > 0:
            loss1 = crit(out1[mask_ex1], labels[mask_ex1])
        else:
            loss1 = torch.tensor(0.0, device=inputs.device, requires_grad=True)
    
        loss2 = torch.tensor(0.0, device=inputs.device, requires_grad=True)
        
        if mask_ex2.sum() > 0:
            inputs_ex2 = inputs[mask_ex2]
            labels_ex2 = labels[mask_ex2]
            
            out2 = model_ref.forward_exit2(inputs_ex2)
            loss2 = crit(out2, labels_ex2)
            
        return loss1, loss2, mask_ex2

    for epoch in range(epochs):
        model.train()
        
        running_metrics = {k: 0.0 for k in metrics}
        total_steps = 0

        loader_iterators = [iter(l) for l in train_iter_loaders]
                
        for _ in range(max_train_batches):
            try:
                batches = [next(it) for it in loader_iterators]
            except StopIteration:
                break
            
            optimizer.zero_grad()

            attack_threshold = model.get_threshold_attack()
            normal_threshold = model.get_threshold_normal()

            thresholds = torch.cat((attack_threshold, normal_threshold), dim=0)
            
            (inputs_a, labels_a) = batches[0]
            (inputs_b, labels_b) = batches[1]
            (inputs_c, labels_c) = batches[2]
            
            inputs_a, labels_a = inputs_a.to(device), labels_a.to(device)
            inputs_b, labels_b = inputs_b.to(device), labels_b.to(device)
            inputs_c, labels_c = inputs_c.to(device), labels_c.to(device)
            total_samples = inputs_a.size(0) + inputs_b.size(0) + inputs_c.size(0)
            
            loss1_a, loss2_a, mask_a_ex2 = process_batch_logic(inputs_a, labels_a, model, thresholds, criterion)
            loss1_b, loss2_b, mask_b_ex2 = process_batch_logic(inputs_b, labels_b, model, thresholds, criterion)
            loss1_c, loss2_c, mask_c_ex2 = process_batch_logic(inputs_c, labels_c, model, thresholds, criterion)

            count_exit2 = mask_a_ex2.sum() + mask_b_ex2.sum() + mask_c_ex2.sum()
                
            loss_ex1_avg = (loss1_a + loss1_b + loss1_c) / 3
            loss_ex2_avg = (loss2_a + loss2_b + loss2_c) / 3
            
            l_joint = loss_ex1_avg * 1 + loss_ex2_avg * 1

            total_loss = l_joint + (count_exit2 / total_samples) * 1
            
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            running_metrics['loss1_a'] += loss1_a.item()
            running_metrics['loss1_b'] += loss1_b.item()
            running_metrics['loss1_c'] += loss1_c.item()
            running_metrics['loss_ex1_avg'] += loss_ex1_avg.item()
            
            running_metrics['loss2_a'] += loss2_a.item()
            running_metrics['loss2_b'] += loss2_b.item()
            running_metrics['loss2_c'] += loss2_c.item()
            running_metrics['loss_ex2_avg'] += loss_ex2_avg.item()
            
            running_metrics['l_joint'] += l_joint.item()
            running_metrics['total_loss'] += total_loss.item()
            
            total_steps += 1

        for key in metrics:
            history['train'][key].append(running_metrics[key] / total_steps)
        
        epoch_train_loss = history['train']['total_loss'][-1]
        epoch_train_jloss = history['train']['l_joint'][-1]
        epoch_train_loss1 = history['train']['loss_ex1_avg'][-1]
        epoch_train_loss2 = history['train']['loss_ex2_avg'][-1]

        model.eval()
        running_metrics_val = {k: 0.0 for k in metrics}
        total_steps_val = 0
        
        val_loader_iterators = [iter(l) for l in val_iter_loaders]
        
        with torch.no_grad():
            for _ in range(max_val_batches):
                try:
                    batches = [next(it) for it in val_loader_iterators]
                except StopIteration:
                    break

                attack_threshold = model.get_threshold_attack()
                normal_threshold = model.get_threshold_normal()

                thresholds = torch.cat((attack_threshold, normal_threshold), dim=0)

                (inputs_a, labels_a) = batches[0]
                (inputs_b, labels_b) = batches[1]
                (inputs_c, labels_c) = batches[2]
  
                inputs_a, labels_a = inputs_a.to(device), labels_a.to(device)
                inputs_b, labels_b = inputs_b.to(device), labels_b.to(device)
                inputs_c, labels_c = inputs_c.to(device), labels_c.to(device)
                total_samples = inputs_a.size(0) + inputs_b.size(0) + inputs_c.size(0)
 
                loss1_a, loss2_a, mask_a_ex2 = process_batch_logic(inputs_a, labels_a, model, thresholds, criterion)
                loss1_b, loss2_b, mask_b_ex2 = process_batch_logic(inputs_b, labels_b, model, thresholds, criterion)
                loss1_c, loss2_c, mask_c_ex2 = process_batch_logic(inputs_c, labels_c, model, thresholds, criterion)

                count_exit2 = mask_a_ex2.sum() + mask_b_ex2.sum() + mask_c_ex2.sum()
                    
                loss_ex1_avg = (loss1_a + loss1_b + loss1_c) / 3
                loss_ex2_avg = (loss2_a + loss2_b + loss2_c) / 3
                
                l_joint = loss_ex1_avg * 1 + loss_ex2_avg * 1

                total_loss = l_joint + (count_exit2 / total_samples) * 1
                                                                        
                running_metrics_val['loss1_a'] += loss1_a.item()
                running_metrics_val['loss1_b'] += loss1_b.item()
                running_metrics_val['loss1_c'] += loss1_c.item()
                running_metrics_val['loss_ex1_avg'] += loss_ex1_avg.item()
                
                running_metrics_val['loss2_a'] += loss2_a.item()
                running_metrics_val['loss2_b'] += loss2_b.item()
                running_metrics_val['loss2_c'] += loss2_c.item()
                running_metrics_val['loss_ex2_avg'] += loss_ex2_avg.item()
                
                running_metrics_val['l_joint'] += l_joint.item()
                running_metrics_val['total_loss'] += total_loss.item()

                
                total_steps_val += 1

        for key in metrics:
            history['val'][key].append(running_metrics_val[key] / total_steps_val)

        epoch_val_loss = history['val']['total_loss'][-1]
        epoch_val_jloss = history['val']['l_joint'][-1]
        epoch_val_loss1 = history['val']['loss_ex1_avg'][-1]
        epoch_val_loss2 = history['val']['loss_ex2_avg'][-1]
        
        attack_threshold_print = model.get_threshold_attack().item()
        normal_threshold_print = model.get_threshold_normal().item()
        
        print(f'Epoch [{epoch+1}/{epochs}] | Train Total: {epoch_train_loss:.4f} | Val Total: {epoch_val_loss:.4f}\nTrain Joint: {epoch_train_jloss:.4f} | Train Loss Exit 1: {epoch_train_loss1:.4f} | Train Loss Exit 2: {epoch_train_loss2:.4f}\nVal Joint: {epoch_val_jloss:.4f} | Val Loss Exit 1: {epoch_val_loss1:.4f} | Val Loss Exit 2: {epoch_val_loss2:.4f}\nAttack Threshold: {attack_threshold_print:.4f} | Normal Threshold: {normal_threshold_print:.4f}\n\n')        
        
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early stopping...")
                if best_model_state: model.load_state_dict(best_model_state)
                break
                
        scheduler.step(epoch_val_loss)
                
    epochs_range = range(1, len(history['train']['l_joint']) + 1)
    
    fig, axs = plt.subplots(1, 3, figsize=(20, 6))
    
    ax = axs[0]
    ax.set_title("Exit 1")
    ax.plot(epochs_range, history['train']['loss1_a'], label='Tr A', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss1_b'], label='Tr B', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss1_c'], label='Tr C', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss_ex1_avg'], label='Tr Avg', linewidth=2)
    ax.plot(epochs_range, history['val']['loss1_a'], label='Val A', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss1_b'], label='Val B', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss1_c'], label='Val C', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss_ex1_avg'], label='Val Avg', color='black', linestyle='--', linewidth=2)
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.legend()
    ax.grid(True)

    ax = axs[1]
    ax.set_title("Exit 2")
    ax.plot(epochs_range, history['train']['loss2_a'], label='Tr A', color='blue', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss2_b'], label='Tr B', color='green', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss2_c'], label='Tr C', color='red', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss_ex2_avg'], label='Tr Avg', color='black', linewidth=2)
    ax.plot(epochs_range, history['val']['loss2_a'], label='Val A', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss2_b'], label='Val B', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss2_c'], label='Val C', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss_ex2_avg'], label='Val Avg', color='black', linestyle='--', linewidth=2)
    ax.set_xlabel('Epochs')
    ax.legend()
    ax.grid(True)

    ax = axs[2]
    ax.set_title("Global Optimization")
    ax.plot(epochs_range, history['train']['l_joint'], label='Tr Joint', color='purple')
    ax.plot(epochs_range, history['train']['total_loss'], label='Tr Total (Joint + (NEx2 / N))', color='orange', linewidth=2)
    ax.plot(epochs_range, history['val']['l_joint'], label='Val Joint', color='purple', linestyle='--')
    ax.plot(epochs_range, history['val']['total_loss'], label='Val Total', color='orange', linestyle='--', linewidth=2)
    ax.set_xlabel('Epochs')
    ax.legend()
    ax.grid(True)

    plt.tight_layout()
    plt.show()
    
    return current_threshold

In [40]:
def evaluate_model(model, loader, confidence_threshold, device):
    model.to(device)
    model.eval()

    total_samples = len(loader.dataset)
    all_predictions = []
    all_labels = []
    exited_early_count = 0
    total_inference_time = 0

    with torch.no_grad():
        for samples, labels in loader:
            samples, labels = samples.to(device), labels.to(device)

            torch.cuda.synchronize()
            start_time = time.perf_counter()
            
            branch_output = model.forward_exit1(samples)
            
            branch_prob = F.softmax(branch_output, dim=1)
            trusts, branch_preds = torch.max(branch_prob, 1)

            batch_predictions = torch.zeros_like(labels)
            
            early_exit_mask = trusts > confidence_threshold
            
            if early_exit_mask.any():
                batch_predictions[early_exit_mask] = branch_preds[early_exit_mask]
                exited_early_count += early_exit_mask.sum().item()

            main_branch_mask = ~early_exit_mask
            if main_branch_mask.any():
                
                samples_to_main = samples[main_branch_mask]
                
                main_output = model.forward_exit2(samples_to_main)
                
                main_prob = F.softmax(main_output, dim=1)
                _, main_preds = torch.max(main_prob, 1)
                
                batch_predictions[main_branch_mask] = main_preds

            torch.cuda.synchronize()
            end_time = time.perf_counter()
            total_inference_time += (end_time - start_time)

            all_predictions.append(batch_predictions.cpu())
            all_labels.append(labels.cpu())

    final_predictions = torch.cat(all_predictions)
    y_data = torch.cat(all_labels)

    correct = (final_predictions == y_data).sum().item()
    accuracy = 100 * correct / total_samples
    exit_rate = 100 * exited_early_count / total_samples
    avg_time_ms = (total_inference_time / total_samples) * 1000

    cm = confusion_matrix(y_data.numpy(), final_predictions.numpy())

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Normal', 'Ataque'],
                yticklabels=['Normal', 'Ataque'])
    plt.xlabel('Rótulo Previsto')
    plt.ylabel('Rótulo Verdadeiro')
    plt.title(f'Matriz de Confusão (Limiar de Confiança = {confidence_threshold})')
    plt.show()

    tn, fp, fn, tp = cm.ravel()
    
    f1 = f1_score(y_data.numpy(), final_predictions.numpy())
    
    tpr = recall_score(y_data.numpy(), final_predictions.numpy())

    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    print(f"True Positives (TP): {tp}")
    print(f"True Negatives (TN): {tn}")
    print(f"False Positives (FP): {fp}")
    print(f"False Negatives (FN): {fn}\n")
    
    print(f"F1 Score: {f1:.4f}")
    print(f"True Positive Rate (TPR) / Recall: {tpr:.4f}")
    print(f"True Negative Rate (TNR) / Specificity: {tnr:.4f}")

    return {
            'accuracy': accuracy,
            'exit_rate': exit_rate,
            'avg_inference_time_ms': avg_time_ms,
            'exited_early_count': exited_early_count,
            'total_samples': total_samples,
            'f1': f1
        }

In [41]:
modelname = 'teste_2_limiar'
modelname

'teste_2_limiar'

In [41]:
epochs = 500

limiar = train_model(
    model, 
    train_loaders, 
    val_loaders, 
    epochs,
    lr=0.0001,
    device=device,
)
torch.save(model.state_dict(), f'models/{modelname}.pth')
print(f"\nModelo treinado e salvo em 'models/{modelname}.pth'")

Epoch [1/500] | Train Total: 1.6879 | Val Total: 1.6738
Train Joint: 0.6879 | Train Loss Exit 1: 0.0000 | Train Loss Exit 2: 0.6879
Val Joint: 0.6738 | Val Loss Exit 1: 0.0000 | Val Loss Exit 2: 0.6738
Attack Threshold: 0.7500 | Normal Threshold: 0.7500


Epoch [2/500] | Train Total: 1.6474 | Val Total: 1.6151
Train Joint: 0.6474 | Train Loss Exit 1: 0.0000 | Train Loss Exit 2: 0.6474
Val Joint: 0.6151 | Val Loss Exit 1: 0.0000 | Val Loss Exit 2: 0.6151
Attack Threshold: 0.7500 | Normal Threshold: 0.7500


Epoch [3/500] | Train Total: 1.5897 | Val Total: 1.5566
Train Joint: 0.5897 | Train Loss Exit 1: 0.0000 | Train Loss Exit 2: 0.5897
Val Joint: 0.5566 | Val Loss Exit 1: 0.0000 | Val Loss Exit 2: 0.5566
Attack Threshold: 0.7500 | Normal Threshold: 0.7500


Epoch [4/500] | Train Total: 1.5537 | Val Total: 1.5182
Train Joint: 0.5537 | Train Loss Exit 1: 0.0000 | Train Loss Exit 2: 0.5537
Val Joint: 0.5182 | Val Loss Exit 1: 0.0000 | Val Loss Exit 2: 0.5182
Attack Threshold: 0.7500 | Nor

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a135c7cdf80>
Traceback (most recent call last):
  File "/home/lucas.sotomaior/Documents/pibic-25-26/myenv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/home/lucas.sotomaior/Documents/pibic-25-26/myenv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 1136, in wait
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/selectors.

In [ ]:
model.load_state_dict(torch.load(f'models/{modelname}.pth'))
print(f"Modelo 'models/{modelname}.pth' carregado\n")

print(f"Base: UNSW")
results = evaluate_model(model, test_loaders[0], limiar, device=device)
print("-" * 20)
print(f"  Accuracy: {results['accuracy']:.4f}%")
print(f"  Avg. Inference Time: {results['avg_inference_time_ms']:.4f} ms")
print(f"  Early Exit Rate: {results['exit_rate']:.4f}% ({results['exited_early_count']}/{results['total_samples']})")
print("-" * 20)

print(f"\nBase: BOT")
results = evaluate_model(model, test_loaders[1], limiar, device=device)
print("-" * 20)
print(f"  Accuracy: {results['accuracy']:.4f}%")
print(f"  Avg. Inference Time: {results['avg_inference_time_ms']:.4f} ms")
print(f"  Early Exit Rate: {results['exit_rate']:.4f}% ({results['exited_early_count']}/{results['total_samples']})")
print("-" * 20)

print(f"\nBase: CIC")
results = evaluate_model(model, test_loaders[2], limiar, device=device)
print("-" * 20)
print(f"  Accuracy: {results['accuracy']:.4f}%")
print(f"  Avg. Inference Time: {results['avg_inference_time_ms']:.4f} ms")
print(f"  Early Exit Rate: {results['exit_rate']:.4f}% ({results['exited_early_count']}/{results['total_samples']})")
print("-" * 20)